In [1]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression as LR
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier as GB
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier as KNC
from Database import Database
from numpy import array
from sklearn.naive_bayes import MultinomialNB as NB
import sqlite3
import tqdm
import random

In [2]:
conn = sqlite3.connect('berita.db')
c = conn.cursor()
c.execute("SELECT Text,Date,Clock,Sentiment From Berita ")
result = c.fetchall()

In [3]:
d = Database()
data = {}
label = {}
for I in result:
    session = d.cariSesi(I[1],I[2])[0]
    if session not in data:
        data[session] = []
        label[session] = I[3]
    sentence = I[0]
    newSentence = ""
    for J in sentence:
        if J.isalpha():
            newSentence += J
        else:
            newSentence += " "
    data[session].append(newSentence)

In [4]:
# Sorting the chronological order

#MeTooLazySoMeBubbleSort

chronology = list(data.keys())

for I in range(len(data.keys())):
    for J in range(I+1,len(data.keys())):
        if chronology[I] > chronology[J]:
            chronology[I],chronology[J] = chronology[J],chronology[I]

In [5]:
len(chronology)

492

In [6]:
def balancedTrain(X,y,mode):
    '''
    mode has two options:
    -> 'CV'
    then balancedTrain would suit it undersampling method for 
    cross validation
    
    -> 'Boosting'
    then balancedTrain would suit is sampling method for
    boosting    
    '''
    assert mode == 'CV' or mode == 'Boosting'
    balancedX = []
    balancedY = []
    
    index = {}
    
    assert len(X) == len(y)
    
    #Mencatat indeks mana yang positif dan mana yang negatif
    for I in range(len(X)):
        if y[I] not in index:
            index[y[I]] = []
        index[y[I]].append(I)
    
    minimumPoint = min([len(I) for I in index.values()])    
    
    #Memastikan jumlah (+) dan (-) sama
    for I in index:
        if mode == 'CV':
            chosen = random.sample(index[I],minimumPoint)
        else:
            chosen = random.choices(index[I],k=minimumPoint)
        for J in chosen:
            balancedX.append(X[J])
            balancedY.append(y[J])
    return balancedX, balancedY

In [7]:
def splitGroup(kFold):
    #Me too lazy to plug in sklearn
    assert len(X) == len(y)
    index = [int(I) for I in range(len(X))]
    random.shuffle(index)
    group = [index[len(index)*I//kFold:len(index)*(I+1)//kFold] for I in range(kFold)]
    return group

In [8]:
#Custom class for GradientBoosting
class Boosting():
    def __init__(self):
        self.clf = GB()
    def fit(self,X,y):
        self.clf.fit(X,y)
    def predict(self,X):
        m = int(X.shape[0] ** (0.5))
        pred = []
        for I in range(m):
            pred.extend(self.clf.predict(X[I*X.shape[0]//m:(I+1)*X.shape[0]//m].toarray()))
        return pred
#Custom class for K Nearest Neighbor
class Neighbors:
    def __init__(self):
        self.clf = KNC()
    def fit(self,X,y):
        self.clf.fit(X,y)
    def predict(self,X):
        m = int(X.shape[0] ** (0.5))
        pred = []
        for I in range(m):
            pred.extend(self.clf.predict(X[I*X.shape[0]//m:(I+1)*X.shape[0]//m]))
        return pred

In [9]:
clfOption = [Boosting(),LR(n_jobs = -1),NB(),LinearSVC(),Neighbors(),RFC()]
mrePred = []

In [10]:
def mrc(pred,Y):
    
    pred = array(pred)
    Y    = array(Y)
    
    TP, FP , TN, FN = 0,0,0,0
    
    for I in range(len(pred)):
        if pred[I] == Y[I]:
            if pred[I] == 1:
                TP += 1
            else:
                TN += 1
        else:
            if pred[I] == -1:
                FP += 1
            else:
                FN += 1
    #print(TP,FP,TN,FN)
    try:
        return ((TP*TN)-(FP*FN)) / ((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**(0.5)
    except:
        return 0

In [22]:
mrePred = []
for index in range(5):
    mreTotal = []
    trainChronology = chronology[:len(chronology)*(index+1)//6]
    testChronology  = chronology[len(chronology)*(index+1)//6:len(chronology)*(index+2)//6]
    
    # Do cross-validation to choose the best feature selection
    
    X = []
    y = []
    
    for I in trainChronology:
        X.extend(data[I])
        y.extend([label[I] for J in range(len(data[I]))])
        
    group = splitGroup(5)
    XkFold = [[X[J] for J in K]for K in group]
    YkFold = [[y[J] for J in K]for K in group]
    counterList = []
    selectList = []
    mreTotal = []

    xTrain = []
    yTrain = []
    
    xTest  = []
    yTest = []
    
    for J in range(5):
        if J == index:
            for L in XkFold[J]:
                xTest.append(L)
            yTest.extend(YkFold[J])
        else:
            for L in XkFold[J]:
                xTrain.append(L)
            yTrain.extend(YkFold[J])

    assert len(xTrain) == len(yTrain)
    xTrainNew , yTrainNew = balancedTrain(xTrain,yTrain,'CV')
    counterList.append(CV(ngram_range=(2,2),min_df=5)) 
    trainVector = counterList[-1].fit_transform(xTrainNew)
    testVector  = counterList[-1].transform(xTest)
    
    selectList.append(SelectKBest(chi2, k = 10000))
    
    trainVector = selectList[-1].fit_transform(trainVector,yTrainNew)
    testVector  = selectList[-1].transform(testVector)

    mreTotal.append(0)
    for J in clfOption:
        J.fit(trainVector,yTrainNew)
        prediction = J.predict(testVector)
        mreTotal[-1] += mrc(prediction,yTest)
    
    index = mreTotal.index(max(mreTotal))
    
    mrePred.append({'post':[],
                    'chronological':[]})
    
    xTrainNew = []
    yTrainNew = []
    # Generating boosting data
    for I in range(9):
        X_temp , y_temp = balancedTrain(xTrain,yTrain,'Boosting')
        xTrainNew.append(X_temp)
        yTrainNew.append(y_temp)

    trainVector = [counterList[index].transform(I) for I in xTrainNew]
    testVector  = counterList[index].transform(xTest)
        
    trainVector = [selectList[index].transform(I) for I in trainVector]
    testVector  = selectList[index].transform(testVector)
    
    for I in clfOption:
        predict = array([int(0) for J in yTest])
        for boostingIter in range(9):
            I.fit(trainVector[boostingIter],yTrainNew[boostingIter])
            predict += I.predict(testVector)
        postPredict = [[-1,1][J>0] for J in predict]
        
        dayPredict = []
        dayY = []
        sumIndex = 0
        
        for J in testChronology:
            dayPredict.append([-1,1][sum(postPredict[sumIndex:sumIndex+len(data[J])])>0])
            dayY.append(label[J])
            sumIndex += len(data[J])
            
        mrePred[-1]['post'].append(mrc(postPredict,yTest))
        mrePred[-1]['chronological'].append(mrc(dayPredict,dayY))
            

In [18]:
name = ["Gradient Boosting","Logistic Regression","Naive Bayes","Linear SVC","K nearest neighbor","Random forest"]
for I in range(len(name)):
    chronological = [J['chronological'][I] for J in mrePred]
    post = [J['post'][I] for J in mrePred]
    print("%s -> (chronological) %f (post) %f " % (name[I],sum(chronological)/5,sum(post)/5))

Gradient Boosting -> (chronological) 0.000000 (post) 0.124020 
Logistic Regression -> (chronological) -0.061229 (post) 0.202800 
Naive Bayes -> (chronological) -0.110458 (post) 0.190508 
Linear SVC -> (chronological) -0.052251 (post) 0.191642 
K nearest neighbor -> (chronological) -0.053863 (post) 0.098195 
Random forest -> (chronological) -0.142782 (post) 0.201578 


In [19]:
mrePred

[{'chronological': [0,
   -0.05303907054347037,
   -0.05303907054347037,
   -0.05303907054347037,
   -0.03482128725155604,
   -0.24640617600447118],
  'post': [0.16244933013562737,
   0.25744044451693676,
   0.22454249973527354,
   0.23682180755565807,
   0.14009937015598561,
   0.26276704519378086]},
 {'chronological': [0,
   -0.21051315346275487,
   -0.21051315346275487,
   -0.17697373762072413,
   -0.21051315346275487,
   -0.12806290809781323],
  'post': [0.15502839297979046,
   0.2304764932080748,
   0.2217317783042985,
   0.21634884714136585,
   0.11268659889500089,
   0.21359028464542726]},
 {'chronological': [0,
   -0.051514008348893185,
   -0.08751750525175062,
   0.00715472338179072,
   -0.01698598887128985,
   -0.020371077711738027],
  'post': [0.08505611260926284,
   0.1808865476372513,
   0.17473803912949432,
   0.17544015433191787,
   0.07409245930322331,
   0.17543264700492828]},
 {'chronological': [0,
   0.1099250293768989,
   -0.16067136948810673,
   0.1464481316025307,